<a href="https://colab.research.google.com/github/leandrobarbieri/pydata-book/blob/2nd-edition/Limpeza_e_Prepara%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpeza e preparação
- Tratamento dos dados em branco (remover, usar médias, zerar)
- Remover duplicidades
- Trasformações (dê-para). switch replace
- Renomear indices e colunas
- Discretização de colunas numéricas
- Tratar os outliers
- Converção de tipos mais adequados
- Filtragem
- Agrupamentos

In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt

# configurações de exibição
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_rows = 20
np.random.seed(12345)
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## Tratamento de dados em branco

### Series

In [2]:
# criando uma serie com um elemento nan
strings_serie = pd.Series(["Leandro", "Barbieri", np.nan, "Moraes"])
strings_serie

0     Leandro
1    Barbieri
2         NaN
3      Moraes
dtype: object

In [3]:
# filtrando um elemento nulo, nan
strings_serie.isnull()

0    False
1    False
2     True
3    False
dtype: bool

In [4]:
# removendo elemento nulo nan
strings_serie.dropna()

0     Leandro
1    Barbieri
3      Moraes
dtype: object

In [5]:
# Usando indexação boleana. O comando acima é igual ao comando abaixo
strings_serie[strings_serie.notnull()]

0     Leandro
1    Barbieri
3      Moraes
dtype: object

### DataFrame

In [6]:
dados1 = pd.DataFrame([ [1, 2, 3], [np.nan, np.nan, np.nan], [np.nan, 4, 5] ])
dados1

,0,1,2
0,1.0,2.0,3.0
1,NaN,NaN,NaN
2,NaN,4.0,5.0


#### Removendo dropna

In [7]:
# removendo linhas (padrão axis=0): Remove todas as linhas que tenham pelo menos um NaN
dados1.dropna(axis=0)

,0,1,2
0,1.0,2.0,3.0


In [8]:
# descarta apenas as linhas que tenham TODAS as colunas com NaN
dados1.dropna(how="all")

,0,1,2
0,1.0,2.0,3.0
2,NaN,4.0,5.0


In [9]:
# faz com que todas as linhas da coluna 2 tenham valor nulo para testes
dados1.loc[:, 2] = np.nan

# descartar apenas as colunas (axis=1) que tenham todas as linhas (how="all") com o valor NaN
dados1.dropna(axis=1, how="all")

,0,1
0,1.0,2.0
1,NaN,NaN
2,NaN,4.0


In [10]:
# removendo com limite de quantidade
dados2 = pd.DataFrame(np.random.randn(7,3))
dados2

,0,1,2
0,-0.204708,0.478943,-0.519439
1,-0.555730,1.965781,1.393406
2,0.092908,0.281746,0.769023
3,1.246435,1.007189,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [11]:
# setando alguns valores nulos para testes
# quatro primeiras linhas da segunda coluna
dados2.iloc[:4, 1] = np.nan
# duas primeiras linhas da terceira coluna
dados2.iloc[:2, 2] = np.nan

dados2

,0,1,2
0,-0.204708,NaN,NaN
1,-0.555730,NaN,NaN
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [12]:
# remove apenas as duas primeiras linhas apesar de existir 4 linhas com valor NaN na coluna 1
dados2.dropna(axis=0, thresh=2)

,0,1,2
2,0.092908,NaN,0.769023
3,1.246435,NaN,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


#### Preenchimento fillna

In [13]:
# preenche com o valor zero os dados np.nan
dados2.fillna(0)

,0,1,2
0,-0.204708,0.000000,0.000000
1,-0.555730,0.000000,0.000000
2,0.092908,0.000000,0.769023
3,1.246435,0.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [14]:
# definindo nomes para as colunas
dados2.columns = ["Primeira", "Segunda", "Terceira"]

# passa um dict com o nome de cada coluna e o valor padrão para ser usado caso seja encontrado um valor np.nan
dados2.fillna({"Primeira": 0, "Segunda": 888, "Terceira": 999})

,Primeira,Segunda,Terceira
0,-0.204708,888.000000,999.000000
1,-0.555730,888.000000,999.000000
2,0.092908,888.000000,0.769023
3,1.246435,888.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


In [15]:
# fillna retorna um novo df alterado, para alterar o df original
# usar inplace=True
dados2.fillna({"Primeira": 0, "Segunda": 888, "Terceira": 999}, inplace=True)
dados2

,Primeira,Segunda,Terceira
0,-0.204708,888.000000,999.000000
1,-0.555730,888.000000,999.000000
2,0.092908,888.000000,0.769023
3,1.246435,888.000000,-1.296221
4,0.274992,0.228913,1.352917
5,0.886429,-2.001637,-0.371843
6,1.669025,-0.438570,-0.539741


#### Interpolação ffill

In [16]:
# metodos de interpolação, preencher para frente ou para trás
dados3 = pd.DataFrame(np.random.randn(6,3), columns=["Pri", "Seg", "Ter"])
dados3

,Pri,Seg,Ter
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.000940,1.343810
3,-0.713544,-0.831154,-2.370232
4,-1.860761,-0.860757,0.560145
5,-1.265934,0.119827,-1.063512


In [17]:
# criando alguns dados em np.nan para testes
dados3.iloc[2:, 1] = np.nan
dados3.iloc[4:, 2] = np.nan
dados3

,Pri,Seg,Ter
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,NaN,1.343810
3,-0.713544,NaN,-2.370232
4,-1.860761,NaN,NaN
5,-1.265934,NaN,NaN


In [18]:
# preenche com o ultimo dado as linhas na sequencia para baixo ffill
# limit=2 limita o preenchimento apenas nas proximas 2 linhas apos o primeiro NAN
dados3.fillna(method="ffill", limit=2)

,Pri,Seg,Ter
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614
2,0.523772,0.124121,1.343810
3,-0.713544,0.124121,-2.370232
4,-1.860761,NaN,-2.370232
5,-1.265934,NaN,-2.370232


In [20]:
# preencher com a média da coluna
# Todas as colunas numéricas: dados3.fillna(dados3.mean())
dados3["Seg"].fillna(dados3["Seg"].mean())

0    3.248944
1    0.124121
2    1.686533
3    1.686533
4    1.686533
5    1.686533
Name: Seg, dtype: float64

## Trasnformação dos dados

#### Removendo duplicados

In [21]:
# criando um dataframe de exemplo
dados4 = pd.DataFrame({"k1": ["um","dois"] * 3 + ["dois"], "k2": [1, 2, 1, 2, 1, 2, 2]})
dados4

,k1,k2
0,um,1
1,dois,2
2,um,1
3,dois,2
4,um,1
5,dois,2
6,dois,2


In [22]:
# True se a combinação ja existe nas linhas superiores
dados4.duplicated()

0    False
1    False
2     True
3     True
4     True
5     True
6     True
dtype: bool

In [23]:
# mantém apenas os não duplicados, apaga a partir da segunda ocorrência. Mantém as primeiras ocorrências
dados4.drop_duplicates()

,k1,k2
0,um,1
1,dois,2


In [24]:
# podemos detectar as duplicidades usando um subconjunto de colunas
# usando a coluna k1 e preservando apenas a ultima ocorrencia (o ultimo "Um" da lista e o ultimo "Dois" da lista)
dados4.drop_duplicates(["k1"], keep="last")

,k1,k2
4,um,1
6,dois,2


#### Dê-Para

In [25]:
# DataFrame com uma coluna de comidas e um valor total
dados5 = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'],
                      'total': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
dados5

,food,total
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [26]:
# Dicionário que será usado para fazer o de-para entre a comida e o tipo de carne
comida_carne = {
    'bacon': 'pig', 
    'pulled pork': 'pig', 
    'pastrami': 'cow', 
    'corned beef': 'cow', 
    'honey ham': 'pig', 
    'nova lox': 'salmon'
}

In [27]:
# lista de comidas padronizada para evitar problemas de upper case no mapeamento
lista_comidas = dados5["food"].str.lower()

# criar uma coluna com o tipo de carne de cada animal
# mapeia cada comida (lista_comidas) com as chaves do dicionario (comida_carne). inner join
dados5["tipo_carne"] = lista_comidas.map(comida_carne)

dados5

,food,total,tipo_carne
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [ ]:
# outra forma de fazer o mapeamento entre os valores da coluna com os valores do dict
dados5["food"].map(lambda x: comida_carne[x.lower()])

In [28]:
# Altera um valor por outro fazendo um de-para
# fillna é um caso específico de substituição
dados4.replace({"um": "Primeiro", "dois": "Segundo"}, inplace=True)
dados4.replace({1: "1ª", 2: "2º"}, inplace=True)

dados4

,k1,k2
0,Primeiro,1ª
1,Segundo,2º
2,Primeiro,1ª
3,Segundo,2º
4,Primeiro,1ª
5,Segundo,2º
6,Segundo,2º


#### Renomeando indices

In [29]:
# alterando o índice das linhas
dados4.index = dados4.index.map(lambda x: x ** 2)
dados4

,k1,k2
0,Primeiro,1ª
1,Segundo,2º
4,Primeiro,1ª
9,Segundo,2º
16,Primeiro,1ª
25,Segundo,2º
36,Segundo,2º


In [30]:
# alterando nome das colunas
dados4.rename(columns=str.upper)

,K1,K2
0,Primeiro,1ª
1,Segundo,2º
4,Primeiro,1ª
9,Segundo,2º
16,Primeiro,1ª
25,Segundo,2º
36,Segundo,2º


In [31]:
# alterando nome das colunas usando um dict
dados4.rename(columns={"k1": "COLUNA1", "k2": "COLUNA2"})

,COLUNA1,COLUNA2
0,Primeiro,1ª
1,Segundo,2º
4,Primeiro,1ª
9,Segundo,2º
16,Primeiro,1ª
25,Segundo,2º
36,Segundo,2º


#### Discretização bins

In [32]:
idades = np.random.randint(0, 100, size=100)
idades.sort()
idades

array([ 1,  2,  3,  4,  5,  5,  5,  5,  7,  8,  8,  9, 11, 12, 12, 12, 13,
       15, 18, 18, 18, 21, 23, 23, 24, 24, 25, 25, 25, 25, 26, 28, 32, 32,
       35, 37, 37, 37, 39, 41, 42, 43, 43, 44, 45, 46, 48, 51, 53, 55, 56,
       59, 59, 60, 60, 61, 62, 62, 62, 64, 64, 65, 65, 65, 69, 69, 69, 70,
       71, 72, 72, 75, 77, 77, 77, 78, 79, 82, 83, 83, 83, 84, 84, 86, 86,
       86, 88, 89, 90, 90, 92, 93, 94, 95, 96, 96, 96, 96, 97, 98])

In [33]:
# definindo as faixas
bins = [0, 18, 35, 65, 100]

# Compartimentos criados: [(0, 18] < (18, 35] < (35, 65] < (65, 100]]
# rigth controla o lado do compartimento que é incluído
categorias = pd.cut(idades, bins=bins, right=True, 
                    labels=["Faixa de 0 a 18 anos", "Faixa de 19 a 35 anos", "Faixa de 36 a 65 anos", "Faixa de 66 a 100 anos"])

# parenteses representa que está incluído no range
categorias

['Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', 'Faixa de 0 a 18 anos', ..., 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos', 'Faixa de 66 a 100 anos']
Length: 100
Categories (4, object): ['Faixa de 0 a 18 anos' < 'Faixa de 19 a 35 anos' < 'Faixa de 36 a 65 anos' <
                         'Faixa de 66 a 100 anos']

In [34]:
# codigo de cada categoria na sequencia das faixas
categorias.codes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int8)

In [35]:
categorias.categories

Index(['Faixa de 0 a 18 anos', 'Faixa de 19 a 35 anos',
       'Faixa de 36 a 65 anos', 'Faixa de 66 a 100 anos'],
      dtype='object')

In [37]:
# quantidade por categoria
categorias.value_counts()

Faixa de 0 a 18 anos      21
Faixa de 19 a 35 anos     14
Faixa de 36 a 65 anos     29
Faixa de 66 a 100 anos    36
dtype: int64

In [38]:
# criando compartimentos com quantidades de itens iguais, passar um valor inteiro que representa a quantidade de compartimentos
# divide a distribuição

# neste caso separa em quatro partes (quartis) com 25% dos elementos
categorias_quartis = pd.qcut(idades, 4)
categorias_quartis.value_counts()

(0.999, 24.0]    26
(24.0, 55.5]     24
(55.5, 77.25]    25
(77.25, 98.0]    25
dtype: int64

## Filtrando outliers

In [39]:
# dados de testes
dados5 = pd.DataFrame(np.random.randn(1000, 4), columns=["Um", "Dois", "Tres", "Quatro"])
dados5

,Um,Dois,Tres,Quatro
0,0.332883,-1.596719,-0.708593,1.544770
1,-0.288232,-1.857159,1.217502,1.403172
2,0.718054,0.067005,0.862797,-0.955341
3,0.042841,0.326556,-0.859770,-0.787531
4,-0.100442,-0.622144,0.958238,0.794095
...,...,...,...,...
995,-1.243393,-0.387105,-0.659995,1.172514
996,-1.249906,1.769541,1.519160,-0.140340
997,1.656912,0.006151,-0.500004,-1.310766
998,-0.832910,-0.055174,-1.444944,0.944028


In [40]:
# exibe estatíticas que ajudam a identificar se existem dados muito fora da distribuição
# por exemplo muito acima da média, ou abaixo
dados5.describe().T

,count,mean,std,min,25%,50%,75%,max
Um,1000.0,-0.033069,0.989949,-3.187784,-0.669611,-0.018139,0.635164,3.513777
Dois,1000.0,0.001834,1.006886,-2.934568,-0.653901,-0.025089,0.670842,3.389123
Tres,1000.0,-0.034493,0.997871,-3.812984,-0.679426,-0.075966,0.664632,2.718222
Quatro,1000.0,0.018250,0.985691,-2.695522,-0.657990,0.013495,0.692102,3.363466


In [41]:
# Identifica as colunas que tem valores nulos
dados5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Um      1000 non-null   float64
 1   Dois    1000 non-null   float64
 2   Tres    1000 non-null   float64
 3   Quatro  1000 non-null   float64
dtypes: float64(4)
memory usage: 31.4 KB


In [42]:
# filtrando dados nulos
dados5.dropna(axis=0, how="all")

,Um,Dois,Tres,Quatro
0,0.332883,-1.596719,-0.708593,1.544770
1,-0.288232,-1.857159,1.217502,1.403172
2,0.718054,0.067005,0.862797,-0.955341
3,0.042841,0.326556,-0.859770,-0.787531
4,-0.100442,-0.622144,0.958238,0.794095
...,...,...,...,...
995,-1.243393,-0.387105,-0.659995,1.172514
996,-1.249906,1.769541,1.519160,-0.140340
997,1.656912,0.006151,-0.500004,-1.310766
998,-0.832910,-0.055174,-1.444944,0.944028


In [43]:
# filtrando valores acima de 3 e abaixo de -3 (corte)
dados5[np.abs(dados5) < 3].describe().T

,count,mean,std,min,25%,50%,75%,max
Um,997.0,-0.030434,0.975279,-2.900738,-0.664425,-0.018020,0.635139,2.963221
Dois,997.0,-0.007897,0.992592,-2.934568,-0.654779,-0.027631,0.668588,2.925225
Tres,997.0,-0.023983,0.980615,-2.848962,-0.676430,-0.074842,0.669622,2.718222
Quatro,997.0,0.008382,0.970576,-2.695522,-0.658604,0.007275,0.689393,2.970426


In [45]:
# somente os registros que estão acima de 3 e abaixo de 3. São as 11 linhas removidas na filtragem anterior
dados5[(np.abs(dados5) > 3).any(1)]

,Um,Dois,Tres,Quatro
166,0.210226,3.216770,-0.912121,-0.907541
257,-0.958620,0.267005,-3.105453,1.257237
353,-0.946487,3.100805,-1.320509,1.720393
369,0.552412,-1.451778,-3.812984,-0.461761
431,3.513777,1.923768,-0.667283,-0.262455
584,0.037474,0.601002,0.803782,3.363466
646,-3.051999,-1.828086,-0.349763,0.285649
790,-3.187784,-2.052495,-0.676430,0.372812
828,1.350441,-0.949437,-3.664025,2.046295
843,-0.070477,-0.175003,1.122770,3.278531


### Amostra aleatória

In [ ]:
# reordenar aleatoriamente as linhas de um df
